# Timesketch Notebook
This is a base notebook for connecting to a dev instance of Timesketch.

## Import
To start a notebook we import few base libraries.
Execute the cell below by pressing the play button or using "shift + enter"

In [99]:
from timesketch_api_client import search
from picatrix.lib import state as state_lib


import altair as alt
import numpy as np
import pandas as pd

state = state_lib.state()

## Select a Sketch.
Now it is time to select a sketch to use, first execute the cell and then change the ID of the sketch to the one you want, and press the button.

In [100]:
generate_connect_button()

## Connect to a sketch

Select a sketch to connect to.

IntText(value=1, description='Sketch ID:')

Button(description='Connect to sketch', style=ButtonStyle())

Output()

*If you want to query data you can use the snippets menu, or create a search obj, and to display a table use `display_table(search_obj.table)` or `display_table(data_frame)`*

To see a list of available helper functions run `%picatrixhelpers` in a cell, or to see a list of functions/magics use `%picatrixmagics`.

## Hunting queries

This chapter contains the hunting queries to find traces related to the used techniques by the threat actor. The queries use forensics artefacts where traces were found during the investigation of Hunt & Hackett, it is possible that other forensics artefacts can be used to find the same traces in different cases or environments depending on the research material. 

### Remote interactive login to Domain Controller  - [T1078.002](https://attack.mitre.org/techniques/T1078/002/), [T1021.001](https://attack.mitre.org/techniques/T1021/001/)

This query can be used to search for deviant logins to a Domain Controller based on the used user account or source IP adress. The naming scheme of the Domain Controller(s) can be adjusted in the query.

In [101]:
search_obj = search.Search(sketch)
search_obj.query_string = 'data_type:"windows:evtx:record" AND event_identifier:4624 AND logon_type:RemoteInteractive AND computer_name:*DC*'
search_obj.return_fields = ('datetime, computer_name, username, source_address')
login_data = search_obj.table
display_table(login_data)

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [{'name'…

### Network login to Domain Controller by the Administrator account - [T1078.001](https://attack.mitre.org/techniques/T1078/001/) 
This query can be used to search for deviant logins by the Administrator account that based on the environment should not be used accross a network. 

In [102]:
search_obj = search.Search(sketch)
search_obj.query_string = 'data_type:"windows:evtx:record" AND event_identifier:4624 AND logon_type:network AND username:administrator'
search_obj.return_fields = ('datetime, computer_name, username, source_address')
login_data = search_obj.table
display_table(login_data)

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [{'name'…

### Scheduled Task related to MEGAsync -  [T1053.005](https://attack.mitre.org/techniques/T1053/005/), [T1567.002](https://attack.mitre.org/techniques/T1567/002/)
This query can be used to find the MEGAsync Update task.

In [103]:
search_obj = search.Search(sketch)
search_obj.query_string = 'data_type:"task_scheduler:task_cache:entry" AND task_name:"MEGAsync Update Task"'
search_obj.return_fields = ('datetime, task_name')
scheduled_tasks = search_obj.table
display_table(scheduled_tasks)

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [{'name'…

### Scheduled task related to the ransomware of the threat actor - [T1486](https://attack.mitre.org/techniques/T1486/)
This query can be used to find traces of Scheduled tasks with the word crypt in the name.

In [104]:
search_obj = search.Search(sketch)
search_obj.query_string = 'data_type:"windows:evtx:record" AND (message_identifier:201 OR message_identifier:203) AND message:crypt*'
search_obj.return_fields = ('datetime, message')
scheduled_tasks = search_obj.table
display_table(scheduled_tasks)

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [{'name'…

### McAfee access protection rule alerts -  [T1562.001](https://attack.mitre.org/techniques/T1562/001/) 
This query can be used to find McAfee access protection rule alerts in the Windows Event Log. These alerts include blocked access protection rules and prevention of common programs running files from the Temp folder.

In [105]:
search_obj = search.Search(sketch)
search_obj.query_string = 'data_type:"windows:evtx:record" AND source_name:McLogEvent AND event_identifier:257'
search_obj.return_fields = ('datetime, message')
alerts = search_obj.table
display_table(alerts)

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [{'name'…

### Stopped services before execution of the ransomware of the threat actor - [T1489](https://attack.mitre.org/techniques/T1489)
This query can be used to detect services that the threat actor stopped before executing the ransomware.

In [106]:
search_obj = search.Search(sketch)
search_obj.query_string = 'event_identifier:7036 AND (message:"Volume Shadow Copy" OR message:"Application Experience" OR message:"File Server Storage Reports Manager" OR message:"Microsoft Software Shadow Copy Provider")'
search_obj.return_fields = ('datetime, computer_name, message')
stopped_services = search_obj.table
display_table(stopped_services)

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [{'name'…

### MEGAsync process execution - [T1588.002](https://attack.mitre.org/techniques/T1588/002/)
This query can be used to detect two specific hashes related to MEGAsync.

* MEGAsync.exe - d92522dcaec6a3d22a1b05d8f5c9ebae08ec74da (SHA1)
* MEGAupdater.exe - 4e7578c638d480da1c3b3b3b54f46b153717981d (SHA1)

In [108]:
search_obj = search.Search(sketch)
search_obj.query_string = 'data_type:"windows:registry:amcache" AND (sha1:d92522dcaec6a3d22a1b05d8f5c9ebae08ec74da OR sha1:4e7578c638d480da1c3b3b3b54f46b153717981d)'
search_obj.return_fields = ('datetime, message, product_name, file_description, company_name, sha1')
executions = search_obj.table
display_table(executions)

Grid(columns_fit='auto', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [{'name'…